In [ ]:
from datasets import load_dataset, load_metric
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer
import re

In [2]:
dataset = load_dataset('Saptarshi7/techqa-squad-style', token=True, trust_remote_code=True)
dataset['test'] = dataset.pop('validation')
dataset['test'] = dataset['test'].filter(lambda x: x['answers']['text'] != [])

In [3]:
tokenizer = AutoTokenizer.from_pretrained('tiiuae/falcon-7b-instruct')

In [ ]:
class TechQA(Dataset):
    def __init__(self, ds, prompt):
        self.samples = []
        for row in tqdm(ds):
            context = row['context']
            context_chunks = tokenizer(context, add_special_tokens=False, truncation=True, max_length=1024,
                                       stride=50, return_overflowing_tokens=True)
            true_spans = row['answers']['text']
            question = row['question']
            
            flag = 0
            for chunk in context_chunks['input_ids']:
                decoded_chunk = tokenizer.decode(chunk, clean_up_tokenization_spaces=False)
                for ans in true_spans:
                    if ans in decoded_chunk:
                        flag = 1
                    else:
                        flag = 0
                        break
                if flag == 1:
                    self.samples.append((prompt.format(context=decoded_chunk, question=question), true_spans))
                    break

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]
    
formatted_dataset = TechQA(dataset['test'], 'Context: {context}\n\nQuestion: {question}\n\nAnswer: ')

c=0
for expanded_prompt, true_answers in formatted_dataset:
    d=0
    for ans in true_answers:
        if re.search(fr'{re.escape(ans)}', expanded_prompt, re.IGNORECASE):
            d+=1
    if d==len(true_answers):
        c+=1
c

In [ ]:
dataset

In [ ]:
c=0
for expanded_prompt, true_answers in formatted_dataset:
    d=0
    for ans in true_answers:
        if re.search(fr'{re.escape(ans)}', expanded_prompt, re.IGNORECASE):
            d+=1
    if d == len(true_answers):
        print(expanded_prompt, true_answers)
        c+=1
    else:
        print(expanded_prompt, true_answers)
        break
c

In [ ]:
c=0
for row in dataset['test']:
    d=0
    for ans in row['answers']['text']:
        if re.search(fr'{re.escape(ans)}', row['context'], re.IGNORECASE):
            #print(row['context'], ans)
            #print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>.')
            d+=1
    if d==len(row['answers']['text']):
        c+=1
c